In [1]:
using Cbc, JuMP, NamedArrays

P : the set of time periods for which staffing requirements must be met
    
bi: the minimum number of baristas that must be working during period i ∈ P
    
S: the set of all possible schedules a barista can work (based on union requirements)
    
asi: defines the periods covered by schedule s ∈ S. asi = 1 means the schedule s ∈ S covers period
i ∈ P , and asi = 0 means the schedule s ∈ S does not cover period i ∈ P
    
cs: the cost to assign a barista to schedule s ∈ S
    
fs: indicates if schedule s ∈ S is full-time. fs = 1 means schedule s is full-time, fs = 0 means
schedule s is part-time
        
M : Maximum number of baristas that can be assigned to a part-time schedule

In [ ]:
#Problem 1-1



#Decision Variables
xs >= 0 #schedules made
xi >= 0 #shifts covering schedule


#Objective
minimize
cs #cost to assign a barista

#Constraints
f0 + f1 >= bi #meeting staffing requirements 
#full time workers
xs <= M #full time workers
xi <= M #part time workers


In [ ]:
#Problem 2-1
Decision Variables
x1>=0 #Starter Company 1
x2>=0 #Starter Company 2
x3>=0 #Starter Company 3
x4>=0 #Starter Company 4
x5>=0 #Starter Company 5

Objective
Maximize
6x1+5x2+12x3+13x4+9x5 #NPV investments

Constraints
low_risk <= 20 #low risk constraint (no hard limit)
high_risk <= 8 #high risk constraint (hard limit of 40% of 20)

In [2]:
#Problem 2-2

using JuMP, Cbc
m=Model()

#data
NPV = [6,5,12,13,9]
cash_expenditure=[6 2 8 10 2;
                2 5 9 11 9;
                4 3 10 9 6;
                2 3 4 8 3]
yearlybudget = 20 #(mil)

#decision variables
@variable(m, x[i=1:5] <= 1) #companies

#objective
@objective(m, Max, sum(x[i] * NPV[i] for i in 1:5))

#constraints
@constraint(m, [year in 1:4], sum(x[i] * cash_expenditure[year, i] for i in 1:5) <= yearlybudget)
@constraint(m, sum(x[i] for i in 3:4) <= 0.4 * sum(x[i] for i in 1:5))

set_optimizer(m, Cbc.Optimizer)
optimize!(m)

Presolve 4 (-1) rows, 5 (0) columns and 20 (-5) elements
0  Obj 45 Primal inf 3.4545425 (3)
3  Obj 27.578512
Optimal - objective value 27.578512
After Postsolve, objective 27.578512, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 27.5785124 - 3 iterations time 0.002, Presolve 0.00


In [ ]:
#Problem 3-1

#Formulate a linear program that minimizes the total distance traveled by students. (You
#do not have to solve this model.)

#Decision Variables
(m, x[i=1:5]) #District
(m, x[j=1:3]) #School


#Objective
Minimize
distance traveled by students

m, min, sum(2.7(CE*ES) + 1.4(D*CE) + 2.4(CE*NS) + 1.1(CE*SW)+ 0.5(CE*FW) + .5(WCE*ES) + .7(WCE*D) + 2.9(WCE*NS) + .8(WCE*SW) + 1.9(WCE*FW) + 1.6(CC*ES) + 2(CC*D) +
    .1(CC*NS) + 1.3(CC*SW) + 2.2(CC*FW) #Each district and school, and time to get to each school from each disctrict
    
#Constraints
2.7(CE*ES) + 1.4(D*CE) + 2.4(CE*NS) + 1.1(CE*SW)+ 0.5(CE*FW)<= 3900
.5(WCE*ES) + .7(WCE*D) + 2.9(WCE*NS) + .8(WCE*SW) + 1.9(WCE*FW)<= 3100
1.6(CC*ES) + 2(CC*D) + .1(CC*NS) + 1.3(CC*SW) + 2.2(CC*FW)<= 2100


sum(x[(i, j)] for school in schools) <= 0.7 * sum(x[(i, j)] for school in schools
x[(i, j)] for school in schools) >= 0.3 * sum(x[(i, j)] for school in schools



In [ ]:
#Problem 4-1

Decision Variables
xas #(for a in A and s in S) #districts and schools


Objective
minimize distance
xas[for a in A][s in s]*das #das is distance, which we want to minimize for each district and school

Constraints
xas[for a in A] = ys
xas[for s in S]=na
ys<=bs
gammaa*xas-h*ys<=0
gammaa*xas-l*ys>=0
ys>=K
xas>=0


In [3]:
#Problem 4-2
# sets of scools (S) and areas (A)
S = 1:3 
A = 1:5

# number of students in each district
n = Dict(zip(A,[1200 1000 1700 2000 2500]))

# capacity of each school
b = Dict(zip(S,[3900 3100 2100]))

using NamedArrays
# distances between district/school pairs
d_matrix = [2.7 0.5 1.6;
    1.4 0.7 2.0;
    2.4 2.9 0.1;
    1.1 0.8 1.3;
    0.5 1.9 2.2]
d_NA = NamedArray(d_matrix,(A,S),("district","school"))

# fraction minority students in the district
g = Dict(zip(A,[0.2 0.1 0.85 0.6 0.9]))

# minimum minority fraction in each school
L = 0.25
# maximum minority fraction in each school
U = 0.8

# minimum enrollment in each school
K = 200;

In [4]:
using JuMP, Cbc

m = Model()


@variable(m, x[A,S] >= 0) # total students sent from area to school
@variable(m, y[S] >= 0) # total students sent to each school

@objective(m, Min, sum(x[a,s]*d_NA[a,s] for a in A, s in S))

@constraint(m, def_y[s in S], sum(x[a,s] for a in A) == y[s])
@constraint(m, assign_all[a in A], sum(x[a,s] for s in S) == n[a])
@constraint(m, school_cap[s in S], y[s] <= b[s])
@constraint(m, upper_cap[s in S], sum(g[a]*x[a,s] for a in A) <= U*y[s])
@constraint(m, lower_cap[s in S], sum(g[a]*x[a,s] for a in A) >= L*y[s])
@constraint(m, min_enroll[s in S], y[s] >= K)
set_optimizer(m, Cbc.Optimizer)

optimize!(m)

Presolve 14 (-6) rows, 18 (0) columns and 69 (-6) elements
0  Obj 0 Primal inf 12685.359 (11)
13  Obj 4810
Optimal - objective value 4810
After Postsolve, objective 4810, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 4810 - 13 iterations time 0.002, Presolve 0.00
